In [3]:
from DM.DmClass import *

dm  = DmClass()

dm.load_data('sa', [], 1, 2)

dm.response()

回复 sa


In [2]:
import grpc
from protos.nlp.dm import dm_pb2
from protos.nlp.dm import dm_pb2_grpc

def run():
    #本地测试
    # channel = grpc.insecure_channel('localhost:50051')
    #数字人
    # channel = grpc.insecure_channel('172.31.208.4:68999')
    #客服
    channel = grpc.insecure_channel('172.31.208.4:50035')
    # 调用 rpc 服务
    stub = dm_pb2_grpc.DmStub(channel)
    history = {}
    while True:
        text = input()
        response = stub.GetDm(dm_pb2.DmRequest(text=text, history=history,session_id='1', robot_id = -1))
        print("Dm client received: " + response.answer)
        history = response.history

if __name__ == '__main__':
    run()

 帮我请假


Dm client received: 请确定您的休假项目(1.年假｜2.带薪病假｜3.育儿假｜4.居家办公｜5.事假｜6.病假｜7.婚假｜8.产假)


 1


Dm client received: 请输入休假的开始与结束时间（例如：2020-2-15 上午｜2020-2-15 下午）


 2020-2-15 上午｜2020-2-15 下午


...

 确定


Dm client received: 您的休假事项已提交！


 谢谢


Dm client received: 不客气，能帮到你我就很开心啦，客气客气~


KeyboardInterrupt: Interrupted by user